
# Template — Projet Data Analysis (Entreprise)

> Utilise ce notebook comme **trame standard** pour tous tes projets (Banque, Médical, Transport, etc.).  
> Remplace les sections **TODO** et exécute pas à pas.

---

## Checklist Projet (rappels rapides)
- [ ] Contexte métier reformulé (2–3 phrases)
- [ ] Cible / objectif (classification, régression, etc.)
- [ ] Données chargées & contrôles de base (head/info/describe)
- [ ] Nettoyage (NA, types, doublons, outliers)
- [ ] Visualisations clés (countplot, histogramme, boxplot, scatter, heatmap, lineplot si temps)
- [ ] KPI / réponses aux questions métier
- [ ] **Rapport exécutif** (10 lignes max, en Markdown à la fin)



## 1) Contexte & Objectifs

**Contexte métier (TODO)**  
Décris le problème comme si tu parlais à un manager :  
- Qu'essayes-tu d'expliquer/prédire ?  
- Pourquoi c'est important ? (impact business)

**Objectif et cible (TODO)**  
- Type de problème : classification / régression / séries temporelles / clustering  
- Variable(s) cible(s) : `...`


In [ ]:

# 2) Imports & options
import os, sys, math, json, warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optionnel : Seaborn (pour un style plus rapide)
import seaborn as sns
sns.set_theme(context='notebook', style='whitegrid')

# Helper esthétique simple
def tidy(ax, title='', xlabel='', ylabel=''):
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.tight_layout()



## 3) Chargement des données

**TODO** : renseigner le chemin de tes fichiers (CSV/Parquet) ou la connexion SQL.


In [ ]:

# Exemple CSV
DATA_PATH = 'data.csv'  # TODO: remplace par ton fichier
df = pd.read_csv(DATA_PATH)

print('Shape:', df.shape)
display(df.head())

# Aperçu global
display(df.info())
display(df.describe(include='all').transpose().round(3))



## 4) Contrôle qualité des données

- Valeurs manquantes / doublons  
- Types de données à optimiser  
- Colonnes inutiles (ID techniques, colonnes vides)  


In [ ]:

# Valeurs manquantes
na = df.isnull().sum().sort_values(ascending=False)
display(na[na > 0])

# Doublons
print('Duplicated rows:', df.duplicated().sum())

# Colonnes candidates à supprimer
id_like = [c for c in df.columns if c.lower() in ('id','rowid','customerid','userid','unnamed: 0','unnamed:0')]
print('Colonnes "ID-like":', id_like)

# Exemple: suppression douce (adapter si nécessaire)
df = df.drop(columns=id_like, errors='ignore')



## 5) Nettoyage & Feature Engineering (de base)

- Conversion de types (dates, catégories)  
- Imputation simple des NA (médiane/mode)  
- Création de features utiles (ex: `CA = qty * price`, `month = date.dt.to_period('M')`)  


In [ ]:

# Conversion dates (si colonne date existe)
for col in df.columns:
    if 'date' in col.lower():
        try:
            df[col] = pd.to_datetime(df[col])
        except Exception:
            pass

# Exemple d'imputation simple num/cat
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

for c in num_cols:
    if df[c].isnull().any():
        df[c] = df[c].fillna(df[c].median())

for c in cat_cols:
    if df[c].isnull().any():
        df[c] = df[c].fillna(df[c].mode()[0])

print('Types:
', df.dtypes.head())



## 6) Détection & gestion des outliers

- Méthode IQR ou Z-score  
- Décision métier : supprimer / winsoriser / transformer (log)  


In [ ]:

# Exemple IQR sur une colonne numérique (TODO: remplace 'col_num')
col = 'col_num'  # TODO
if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    low, high = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    outliers = df[(df[col] < low) | (df[col] > high)]
    print(f'Outliers détectés sur {col}:', outliers.shape[0])
    # Exemple: winsorisation douce
    df[col] = np.where(df[col] < low, low, np.where(df[col] > high, high, df[col]))



## 7) EDA — Visualisations essentielles

- Countplot (catégories)  
- Histogramme/KDE (numériques)  
- Box/Violin (numérique vs catégorie)  
- Scatter (numérique vs numérique)  
- Heatmap de corrélations  
- Lineplot (séries temporelles)  


In [ ]:

# 7.1 Répartition d'une catégorie (TODO: remplace 'classe')
if 'classe' in df.columns:
    ax = sns.countplot(x='classe', data=df)
    tidy(ax, title='Répartition de la classe', xlabel='Classe', ylabel='Effectif')
    plt.show()

# 7.2 Distribution numérique (TODO: remplace 'valeur')
if 'valeur' in df.columns and pd.api.types.is_numeric_dtype(df['valeur']):
    ax = sns.histplot(df['valeur'], bins=30, kde=True)
    tidy(ax, title='Distribution de la variable', xlabel='Valeur', ylabel='Fréquence')
    plt.show()

# 7.3 Boxplot num vs cat (TODO: remplace 'classe','valeur_num')
if set(['classe','valeur_num']).issubset(df.columns):
    ax = sns.boxplot(data=df, x='classe', y='valeur_num')
    tidy(ax, title='Valeur par classe', xlabel='Classe', ylabel='Valeur')
    plt.show()

# 7.4 Scatter (TODO: remplace 'x','y')
if set(['x','y']).issubset(df.columns):
    ax = sns.scatterplot(data=df, x='x', y='y', alpha=0.7)
    tidy(ax, title='Relation X vs Y', xlabel='X', ylabel='Y')
    plt.show()

# 7.5 Heatmap corrélations
num = df.select_dtypes(include=[np.number])
if num.shape[1] > 1:
    corr = num.corr()
    plt.figure(figsize=(8,6))
    sns.heatmap(corr, cmap='coolwarm', center=0)
    plt.title('Matrice de corrélation (numériques)')
    plt.tight_layout()
    plt.show()

# 7.6 Lineplot temporel (TODO: remplace 'date','valeur_ts')
if set(['date','valeur_ts']).issubset(df.columns):
    tmp = (df
           .assign(date=pd.to_datetime(df['date']))
           .set_index('date')
           .sort_index()['valeur_ts']
           .resample('M').sum())
    ax = sns.lineplot(x=tmp.index, y=tmp.values)
    tidy(ax, title='Évolution mensuelle', xlabel='Mois', ylabel='Somme')
    plt.show()



## 8) KPI & Questions métier (à adapter)

- **Exemples** :  
  - Taux de churn global et par pays (banque)  
  - CA mensuel, panier moyen, top produits (e-commerce)  
  - Distribution diagnostic et features clés (médical)  


In [ ]:

# Exemples génériques (adapte à ton cas)
# Taux global d'une cible binaire 'target' (0/1)
if 'target' in df.columns:
    rate = df['target'].mean()
    print('Taux global (target=1):', round(rate*100, 2), '%')

# Top N entités par 'valeur' (TODO: remplace 'entite','valeur')
if set(['entite','valeur']).issubset(df.columns):
    top = (df.groupby('entite')['valeur'].sum().nlargest(10).sort_values())
    ax = top.plot(kind='barh', figsize=(8,4))
    tidy(ax, title='Top 10 entités', xlabel='Valeur', ylabel='Entité')
    plt.show()



## 9) (Optionnel) Modèle de base & validation

- Baseline interprétable (régression logistique)  
- Comparaison avec un modèle d'arbres (RandomForest)  
- Métriques adaptées (classification : accuracy, precision, recall, F1, ROC-AUC)  


In [ ]:

# Exemple classification binaire (si 'target' et quelques features numériques existent)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

target_col = 'target'  # TODO
feat_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c != target_col]

if target_col in df.columns and len(feat_cols) >= 1:
    X = df[feat_cols].copy()
    y = df[target_col].copy()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    scaler = StandardScaler(with_mean=False)  # robuste aux sparsités éventuelles
    X_train_s = scaler.fit_transform(X_train)
    X_test_s  = scaler.transform(X_test)

    clf = LogisticRegression(max_iter=1000, class_weight='balanced')
    clf.fit(X_train_s, y_train)

    y_pred = clf.predict(X_test_s)
    y_prob = clf.predict_proba(X_test_s)[:,1]

    print(classification_report(y_test, y_pred, digits=3))
    print('ROC-AUC:', round(roc_auc_score(y_test, y_prob), 3))
else:
    print("👉 Skip modèle : configure 'target' et 'feat_cols' selon ton dataset.")



## 10) Rapport exécutif (à compléter)

**Contexte** :  
- (résumé 2–3 phrases)

**Résultats clés** :  
- …  
- …  
- …  

**Insights & Recommandations** :  
- …  
- …  
- …  

> Garde cette section concise et orientée **décision**. C’est ce que l’on présente aux managers.
